In [3]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from groq import Groq
from dotenv import load_dotenv
import os

load_dotenv('/Users/kiara/Documents/matkassen-etl-pipeline/.env')
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Ladda data med full sökväg
conn = sqlite3.connect('/Users/kiara/Documents/matkassen-etl-pipeline/matkassen-etl/notebook/matkassen.db')
df = pd.read_sql('SELECT * FROM matkassen_tvattad', conn)
conn.close()

# Ta bara rader med omdöme_text
df_sentiment = df[df['omdöme_text'].notna()].copy()
print(f"Antal omdömen att analysera: {len(df_sentiment)}")
df_sentiment['omdöme_text'].head(5)

Antal omdömen att analysera: 1117


1            Älskar att slippa planera middagar. Tack!
2    Svamprisotto var verkligen inte gott. Slängde ...
3           Priset är högt men sparar tid. På gränsen.
4        Fungerar, men recepten börjar bli repetitiva.
9                 Snabb leverans och allt var fräscht.
Name: omdöme_text, dtype: object

In [8]:
def analysera_sentiment(text):
    try:
        response = client.chat.completions.create(
          model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "Du är en sentimentanalysmodell. Svara ENDAST med ett ord: Positiv, Neutral eller Negativ."},
                {"role": "user", "content": f"Klassificera detta omdöme: {text}"}
            ],
            max_tokens=10
        )
        raw = response.choices[0].message.content.strip()
        print(f"Råsvar: '{raw}'")  # tillfällig debug
        return raw
    except Exception as e:
        print(f"Fel: {e}")
        return "Neutral"

# Testa på ett omdöme
test = df_sentiment['omdöme_text'].iloc[0]
print(f"Text: {test}")
print(f"Sentiment: {analysera_sentiment(test)}")

Text: Älskar att slippa planera middagar. Tack!
Råsvar: 'Positiv'
Sentiment: Positiv


In [5]:
print("Analyserar sentiment... (kan ta några minuter)")
df_sentiment['sentiment'] = df_sentiment['omdöme_text'].apply(analysera_sentiment)
print("✅ Klart!")
print(df_sentiment['sentiment'].value_counts())

Analyserar sentiment... (kan ta några minuter)
✅ Klart!
sentiment
Neutral    1117
Name: count, dtype: int64


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sentiment_counts = df_sentiment['sentiment'].value_counts()
colors = {'Positiv': '#2ecc71', 'Neutral': '#f39c12', 'Negativ': '#e74c3c'}
sentiment_counts.plot(kind='bar', ax=axes[0],
                      color=[colors.get(x, 'gray') for x in sentiment_counts.index],
                      edgecolor='black')
axes[0].set_title('Fördelning av Sentiment', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Antal', fontsize=12)
axes[0].tick_params(axis='x', rotation=0)

sentiment_kass = pd.crosstab(df_sentiment['kasstyp'], df_sentiment['sentiment'], normalize='index') * 100
sentiment_kass.plot(kind='bar', ax=axes[1],
                    color=[colors.get(x, 'gray') for x in sentiment_kass.columns],
                    edgecolor='black')
axes[1].set_title('Sentiment per Kasstyp (%)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Andel (%)', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend(title='Sentiment')

plt.tight_layout()
plt.savefig('sentiment_analys.png', dpi=300, bbox_inches='tight')
print("✅ Sparad: sentiment_analys.png")
plt.show()